In [ ]:
from nanomesh.generator import Generator
from nanomesh.utils import generate_mesh_from_binary_image
from nanomesh import Volume
import numpy as np
import math
import pygalmesh

### What is different in this notebook?

What should be exposed from Pore3D to make it work?

In [ ]:
class Pore3D_semi_periodic(pygalmesh.DomainBase):
    def __init__(self):
        super().__init__()
        
        self.gen = Generator(
            a=680, 
            c=math.sqrt(2)*680, 
            radius=0.24*680,
        )

        # Possible rotation/transformation of the coordinate system
        theta = math.pi * 1/180
        c = math.cos(theta)
        s = math.sin(theta)
        trans = np.array([
            [ c, 0, s],
            [ 0, 1, 0],
            [-s, 0, c],
        ])
        
        self.size = (100, 100, 100)
        self.res = (10, 10, 10)
        self.data = self.gen.generate_vect(
            self.size, 
            self.res, 
            transform=trans, 
            bin_val=[0.,1.],
        )
        self.data = self.data[:,:68,:]
                
        self.x = np.linspace(0,1.,self.data.shape[0])
        self.y = np.linspace(0,0.68,self.data.shape[1])
        # self.y = np.linspace(0,1.,self.data.shape[1])
        self.z = np.linspace(0,1.,self.data.shape[2])

        # remove a small top/bottom layer
        self.data[:,:,:10] = 0
        self.data[:,:,-11:] = 0
        
    def eval(self, x):
        ix = np.argmin(np.abs(x[0]-self.x))
        iy = np.argmin(np.abs(x[1]-self.y))
        iz = np.argmin(np.abs(x[2]-self.z))
        if self.data[ix,iy,iz] == 1:
            out = -1.
        else:
            out = 1.
        return out
    

In [ ]:
pore = Pore3D_semi_periodic()

pore_vol = Volume(pore.data)
pore_vol.show_slice(along='z')

In [ ]:
mesh = pygalmesh.generate_periodic_mesh(
    pore,
    [0, 0, 0, 1., 0.68, 1.],
    max_cell_circumradius=0.05,
    min_facet_angle=30,
    max_radius_surface_delaunay_ball=0.05,
    max_facet_distance=0.025,
    max_circumradius_edge_ratio=2.0,
    number_of_copies_in_output=1,
    make_periodic=True,
    # odt=True,
    # lloyd=True,
    verbose=False,
    )

In [ ]:
import pyvista as pv
pv.plot_itk(mesh)

In [ ]:
from nanomesh.periodic_utils import insert_periodic_info
mesh = insert_periodic_info(mesh, [0,0,0,1.,0.68,1.])

In [ ]:
mesh.write('nanopt_semi_period_1domain.gmsh', file_format="gmsh22", binary=False)